In [1]:
#Load the libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
import numpy as np


# Data Loading
Load the dataset and show first few rows. 

In [27]:
# Load the dataset
file_path = './marketing_campaign.csv'
df = pd.read_csv(file_path, sep='\t')
df.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


# Create a Preprocessing Pipeline
 Chain multiple feature engineering and data preprocessing steps together in a single object to ensure that all steps are applied sequentially and that data leakage is avoided.

In [ ]:
# Define column categories
columns_to_drop = ['ID', 'Year_Birth', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Marital_Status', 'Z_CostContact', 'Z_Revenue']
numerical_columns = ['Income', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
                     'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                     'NumWebVisitsMonth', 'Age', 'Total_Children', 'Total_Household_Size', 'Total_Spending',
                     'Total_Purchase_Num', 'Discount_Dependency', 'Loyalty_Months', 'Campaign_Response_Rate',
                     'Shopping_Frequency', 'Recency']
categorical_columns = ['Education', 'Marital_Status_Grouped']

# Function to Remove Outliers
def remove_outliers(df):
    if 'Income' not in df.columns:
        print("Warning: 'Income' column not found. Skipping outlier removal.")
        return df
    income_threshold = df['Income'].quantile(0.999)  # 99.9th percentile
    df_filtered = df[df['Income'] <= income_threshold].reset_index(drop=True)
    print(f"Removed {df.shape[0] - df_filtered.shape[0]} income outliers.")
    return df_filtered

# Define the Preprocessing Pipeline
pipeline = Pipeline(steps=[
    ('outlier_removal', FunctionTransformer(remove_outliers, validate=False)),
    ('feature_engineering', FunctionTransformer(
        lambda df: df.assign(
            Age=2024 - df['Year_Birth'],
            Total_Children=df['Kidhome'] + df['Teenhome'],
            Marital_Status_Grouped=df['Marital_Status'].map({
                'Single': 'Single_Household',
                'Divorced': 'Single_Household',
                'Widow': 'Single_Household',
                'Alone': 'Single_Household',
                'Together': 'Couples',
                'Married': 'Couples',
                'Absurd': 'Unknown',
                'YOLO': 'Unknown'
            }),
        ), validate=False)
    ),
    ('feature_engineering2', FunctionTransformer(
        lambda df: df.assign(
            Total_Household_Size=(df['Marital_Status'].isin(['Together', 'Married'])).astype(int) + 1,  # Removed Total_Children from calculation
            Total_Spending=df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
                              'MntGoldProds']].sum(axis=1),
            Total_Purchase_Num=df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']].sum(axis=1),
        ), validate=False)
    ),
    ('feature_engineering3', FunctionTransformer(
        lambda df: df.assign(
            Discount_Dependency=np.where(
                df['Total_Purchase_Num'] > 0,
                df['NumDealsPurchases'] / df['Total_Purchase_Num'],
                0  # Set to 0 when Total_Purchase_Num is zero
            ),
            Loyalty_Months=((pd.to_datetime('today') - pd.to_datetime(df['Dt_Customer'],
                                                                     format='%d-%m-%Y')).dt.days // 30),
            Campaign_Response_Rate=df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
                                      'Response']].sum(axis=1) / 6,
            Shopping_Frequency=df[['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
                                  'NumStorePurchases']].sum(axis=1)
        ).drop(columns_to_drop, axis=1), validate=False)
    ),
    ('preprocessing', ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numerical_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
        ],
        remainder='passthrough'
    ))
])

In [ ]:
# Fit the pipeline
df_featured = pipeline.fit_transform(df)

# Get column names
transformer = pipeline.named_steps['preprocessing']
numerical_names = numerical_columns
categorical_encoded_names = transformer.named_transformers_['cat'].get_feature_names_out(categorical_columns)

# Combine all names
final_columns = numerical_names + list(categorical_encoded_names)

#Find the columns that were passed through.
passthrough_columns = [col for col in df.columns if col not in numerical_columns and col not in categorical_columns and col not in columns_to_drop]

final_columns = final_columns + passthrough_columns

processed_df = pd.DataFrame(df_featured, columns=final_columns)

processed_df.to_csv('marketing_campaign_processed_with_pipeline.csv', index=False)  # Save to CSV

display(processed_df)

# Store the fitted pipeline as an attribute 
pipeline.pipeline = pipeline

Removed 27 income outliers.


,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,...,Marital_Status_Grouped_Single_Household,Marital_Status_Grouped_Unknown,Recency,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,0.294551,0.976923,1.548034,1.708066,2.452673,1.483369,0.848775,0.358009,1.427545,2.550438,...,1.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.259751,-0.872804,-0.637524,-0.722217,-0.651432,-0.634452,-0.733634,-0.167023,-1.127817,-0.577461,...,1.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.927858,0.357383,0.568301,-0.177654,1.338847,-0.147597,-0.038918,-0.692055,1.427545,-0.229916,...,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.185531,-0.872804,-0.562160,-0.659210,-0.505357,-0.585766,-0.752932,-0.167023,-0.762765,-0.925005,...,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.301836,-0.392586,0.417573,-0.218159,0.151983,-0.001540,-0.559955,1.408073,0.332390,0.117628,...,0.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,0.439542,1.196282,0.417573,0.069875,0.078945,2.213652,3.917106,-0.167023,1.792597,0.117628,...,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2209,0.570715,0.298097,-0.662645,-0.614205,-0.687951,-0.658795,-0.695039,2.458137,1.427545,-0.229916,...,0.0,0.0,56.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2210,0.240173,1.786178,0.543180,0.227393,-0.103649,-0.366682,-0.386276,-0.692055,-0.762765,0.117628,...,1.0,0.0,91.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2211,0.816565,0.363312,0.090995,0.213892,0.772804,0.071488,0.327738,-0.167023,0.697442,0.812717,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
